In [1]:
import requests
import os
from dotenv import load_dotenv
import json

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
API_KEY = os.getenv('API_KEY')
IBM_CLOUD_URL = os.getenv('IBM_CLOUD_URL')
PROJECT_ID = os.getenv('PROJECT_ID')

# Your Watson X agent endpoint
DEPLOYMENT_ENDPOINT = 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/6261f83b-14d4-4666-9243-2cb36bdcd698/ai_service_stream?version=2021-05-01'

def get_auth_token():
    """Get authentication token from IBM Cloud"""
    try:
        token_response = requests.post(
            'https://iam.cloud.ibm.com/identity/token', 
            data={
                "apikey": API_KEY, 
                "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
            }
        )
        token_response.raise_for_status()
        return token_response.json()["access_token"]
    except Exception as e:
        print(f"Error getting authentication token: {e}")
        return None

def chat_with_watson(message, token):
    """Send message to Watson X agent and get response"""
    headers = {
        'Content-Type': 'application/json', 
        'Authorization': f'Bearer {token}'
    }
    
    payload = {
        "messages": [
            {
                "content": message,
                "role": "user"
            }
        ]
    }
    
    try:
        response = requests.post(
            DEPLOYMENT_ENDPOINT,
            json=payload,
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

def main():
    """Main interactive chat function"""
    print("🤖 Watson X Agent Chat")
    print("=" * 50)
    
    # Check if API key is loaded
    if not API_KEY:
        print("❌ Error: API_KEY not found in .env file")
        return
    
    print(f"✅ API Key loaded: {API_KEY[:10]}...")
    print(f"✅ IBM Cloud URL: {IBM_CLOUD_URL}")
    print(f"✅ Project ID: {PROJECT_ID}")
    
    # Get authentication token
    print("\n🔐 Getting authentication token...")
    token = get_auth_token()
    
    if not token:
        print("❌ Failed to get authentication token. Please check your API key.")
        return
    
    print("✅ Authentication successful!")
    print("\n💬 You can now chat with your Watson X agent!")
    print("Type 'quit' or 'exit' to stop\n")
    
    # Interactive chat loop
    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Goodbye!")
                break
            
            if not user_input:
                print("Please enter a message or type 'quit' to exit.")
                continue
            
            # Send message to Watson X agent
            print("🤖 Watson X Agent: ", end="", flush=True)
            response = chat_with_watson(user_input, token)
            
            if response:
                # Parse and display response
                if 'choices' in response and response['choices']:
                    agent_response = response['choices'][0].get('message', {}).get('content', 'No response content')
                    print(agent_response)
                elif 'results' in response:
                    # Alternative response format
                    agent_response = response['results'][0].get('generated_text', 'No response content')
                    print(agent_response)
                else:
                    print("Received response but couldn't parse content:")
                    print(json.dumps(response, indent=2))
            else:
                print("❌ Failed to get response from Watson X agent")
            
            print()  # Add blank line for readability
            
        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ An error occurred: {e}")

if __name__ == "__main__":
    main()


🤖 Watson X Agent Chat
✅ API Key loaded: Q07iDID0og...
✅ IBM Cloud URL: https://us-south.ml.cloud.ibm.com
✅ Project ID: ca4ab8b0-0c39-4985-9400-2df348bbf894

🔐 Getting authentication token...
✅ Authentication successful!

💬 You can now chat with your Watson X agent!
Type 'quit' or 'exit' to stop

🤖 Watson X Agent: Request error: 403 Client Error: Forbidden for url: https://us-south.ml.cloud.ibm.com/ml/v4/deployments/6261f83b-14d4-4666-9243-2cb36bdcd698/ai_service_stream?version=2021-05-01
❌ Failed to get response from Watson X agent

👋 Goodbye!
